In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re

import json

import mysql_connector as connector

import pandas as pd

df = pd.DataFrame(columns=['name', 'sum', 'linc', 'website'])

In [2]:
# Константа основного сайта
def get_main_url():
    return 'https://www.superjob.ru'

# Функция получаем из строки города и пишет их БД
def get_geo_and_save_db():
    #user_agent = {'User-agent': 'Mozilla/5.0'}
    html = requests.get(f'{get_main_url()}/vacancy/search/?keywords=sql&geo%5Bt%5D%5B0%5D=73&geo%5Bt%5D%5B1%5D=4&geo%5Bt%5D%5B2%5D=14&geo%5Bt%5D%5B3%5D=13&geo%5Bt%5D%5B4%5D=33&geo%5Bt%5D%5B5%5D=12&geo%5Bt%5D%5B6%5D=55&geo%5Bt%5D%5B7%5D=106&geo%5Bt%5D%5B8%5D=17&geo%5Bt%5D%5B9%5D=5&geo%5Bt%5D%5B10%5D=173&geo%5Bt%5D%5B11%5D=1090&geo%5Bt%5D%5B12%5D=42&geo%5Bt%5D%5B13%5D=88&geo%5Bt%5D%5B14%5D=2528&geo%5Bt%5D%5B15%5D=1644').text#, headers = user_agent
    conn = connector.conn()
    geo(conn, html)

# функция URL для поиска вакансий
def get_url(**params):
    keywords = ""
    geo = ""
    if params.get("keywords") != None:
        keywords = params["keywords"]
    if params.get("geo") != None:
        geo = f'&geo%5Bt%5D%5B0%5D={params["geo"]}'
        
    return f'{get_main_url()}/vacancy/search/?keywords={keywords}{geo}'

# функция получения массива URL с детализацией вакансии
def  get_page_vakansii(html):
    
    vakancii = html.find_all('a', {'class':'_1QIBo'})

    return [get_main_url() + v.get('href') for v in vakancii] 

# функция получения наименования вакансии со страницы вакансии
def get_name_vakansi(url):
    html = requests.get(url).text
    html = bs(html, 'lxml')
    name_vakansi = html.find_all('h1', {'class':'_3mfro'})
    
    return name_vakansi[0].text  

#Получение реквизитов вакансии
def get_data_vakansii(url):
    html1 = requests.get(url).text
    html1 = bs(html1, 'lxml')
    name_vakansi = html1.find_all('h1', {'class':'_3mfro'})

    sum_vakansi = html1.find_all('span', {'class':'_2Wp8I'})
    sum_vakansi = re.findall('<span>([\d ]+)<\/span>', str(sum_vakansi[0]).replace('\xa0',''))

    #print(name_vakansi[0].text, sum_vakansi[0])

    #df.loc[len(df)+1] = 
    
    sum_vakansi = sum_vakansi if len(sum_vakansi) > 0 else 'По договоренности'
    
    if type(sum_vakansi) == list:
        sum_vakansi = f'от {sum_vakansi[0]} до {sum_vakansi[1]}' if len(sum_vakansi) > 1 else f'от {sum_vakansi[0]}'
        
        
        
    
    return [name_vakansi[0].text, sum_vakansi, url, get_main_url()]

# функция получения городов из кэша
def geo(conn, html = None, geo_name = None):
    if html != None:
        html = bs(html, 'lxml')
        scripts = html.find_all('a', {'class':'bv_II'})
        i = 0
        cites = {}
        for script in scripts:

            href = script.get('href')
            #print(f'{i} {script}')
            if href == None:
                continue

            if href.find('/vacancy/search/',0) < 0:
                continue
            i +=1
            #print(f'{i} {href}')
            city = bs(str(script), 'lxml')
            pattern = '>([А-Яа-я\ \-]+)<'
            string = str(city.find_all('span', {'class':'_3mfro'}))
            city = re.findall(pattern,string)

            if len(city)>0:
                arr_id = []
                pattern = '=([\d]+)'
                ids = re.findall(pattern,str(script))
                for id in ids:
                    arr_id.append(id)
                cites[city[0]] =  arr_id
        #print(f'{cites}')
        
        last_ids = cites[list(cites)[-1]]

        for k, v in cites.items():
            id = list((i for i in last_ids if i not in cites[k]))[0]
            last_ids = cites[k]    
            #print(f'{k} {list(id)[0]}')
            query = f'delete from net.geo where id = "{id}"'
            connector.exec(conn, query, '')
            connector.exec(conn, f'''
            insert into net.geo (id, name)
            select {id} as id, "{k}" as name
            from DUAL
            where not exists(select 1 from net.geo where id = {id})''','')
    if geo_name != None:
        where = '' if geo_name == '' else f'where name = "{geo_name}"'
        cursor = connector.get_date(conn, f'select id, name from net.geo {where}', '')
        return cursor
        

# функция получения следующей страницы
def get_next_page(pagas, url):
    isFind = False
    for page in pages:
        if page.get('href').find("onMap",0) != -1:
            continue
        new_url = get_main_url()+page.get('href')
        if str(url) == new_url:
            isFind = True
        if (str(url) != new_url and isFind):
            return new_url
        #print(page.text)
        
    return None

#Патерн для поиска следующей страницы через re. Может позже пригодится
#patern = 'span class="_3IDf-">([\d]|[Дд][а-яА-Я]+)'

# Функция формирует массив страниц
# используем ее для последующего обхода
def get_pages(pages,url):

    urls = [url]
    while 1==1:

        next_url = get_next_page(pages,urls[len(urls)-1])

        if next_url not in urls:
            urls.append(next_url)
        else:
            break
    return urls

#у нас пока мало городов для поиска, но мы стремимся к большему
# выбирите город в котором необходимо искать вакансии
conn = connector.conn()
cities = dict(geo(conn, None, ""))

print(cities)

inp = input('''

Выбор города. 
Введите наименование из списка выше: ''')
inp

#ищем в БД
cities = dict(geo(conn, None, inp))


#если mysql не подключен то закоментарить все с верху до текущей строки и в 

city = list(cities)[0]
if len(cities) > 0:
    #Запрос в sj
    url = get_url(keywords = "sql", geo = city)#(вакансия , город)
    html = requests.get(url).text
    html = bs(html, 'lxml')

    #в url у нас основная страница отражающая результат поиска а также ссылки на страницы остальных результатов поиска
    #соберем сначала все страницы в массив , а далее уже пройдем по массиву и обработаем рузультиаты поиска
    pages = html.find_all('a', {'class':'_3ze9n'})

    for page_search in get_pages(pages,url):
        print(page_search)
        if page_search == None:
            continue
        html = requests.get(page_search).text
        html = bs(html, 'lxml')
        #обход страниц вакансий и запись их в датафрейм
        for vacancy in get_page_vakansii(html):
            df.loc[len(df)+1] = get_data_vakansii(vacancy)
# выводим результат в dataframe
df

{73: 'Ростов-на-Дону', 4: 'Москва', 14: 'Санкт-Петербург', 13: 'Новосибирск', 33: 'Екатеринбург', 12: 'Нижний Новгород', 55: 'Казань', 1090: 'Нурлат', 106: 'Челябинск', 17: 'Омск', 5: 'Самара', 173: 'Уфа', 42: 'Воронеж', 88: 'Владимир', 2528: 'Владимиро-Александровское', 1644: 'Владимир-Волынский'}


Выбор города. 
Введите наименование из списка выше: Уфа
https://www.superjob.ru/vacancy/search/?keywords=sql&geo%5Bt%5D%5B0%5D=173
None


,name,sum,linc,website
1,"Программист (Lexema, C#)",от 30000,https://www.superjob.ru/vakansii/programmist-2...,https://www.superjob.ru
2,Специалист по внедрению и сопровождению ИС,от 25000,https://www.superjob.ru/vakansii/specialist-po...,https://www.superjob.ru
3,Инженер-программист Delphi,По договоренности,https://www.superjob.ru/vakansii/inzhener-prog...,https://www.superjob.ru
4,Инженер-программист,от 20000 до 100000,https://www.superjob.ru/vakansii/inzhener-prog...,https://www.superjob.ru
